# Ball-sphere image processing
Given photos of the ball, extract spherical pathes on the same position of the photo, then stitch into 1 image

In [132]:
import numpy as np
import matplotlib.pyplot as plt
# from matplotlib import rcParams
import cv2

### Read and Plot by cv2
refer to the Link-course "OpenCV Python Developers"

In [133]:
## read photo
path0 = r'C:\\Users\\WANGH0M\\Desktop\\opencv\\ball_photos'
path = r'C:\\Users\\WANGH0M\\Desktop\\ball_drillbit\\ball\\0'
name = '\\0B4A4651.jpg'
img = cv2.imread(path+name,1)
print(img.shape, img.size)

## change jpg to png
cv2.imwrite(path + "\\1.png", img)
img = cv2.imread(path+'\\1.png',1)

## rescale the photo
img = cv2.resize(img, (0,0), fx=0.1, fy=0.1)

## plot the photo
# cv2.imshow("Original", img)
# cv2.waitKey()
# cv2.destroyAllWindows()

## crop the photo
num_row, num_col = img.shape[0], img.shape[1]
subimg = img[:, int(num_col*0.2):int(num_col*0.8), :]

cv2.imshow("Sub-image", subimg)
cv2.waitKey()
cv2.destroyAllWindows()

(5464, 8192, 3) 134283264


In [134]:
## convert it to a grayscale image
gray_img = cv2.cvtColor(subimg, cv2.COLOR_BGR2GRAY)

## display the image
# cv2.imshow("Gray Image", gray_img)
# cv2.waitKey()
# cv2.destroyAllWindows()


In [135]:
def circle_moment(subimg, gray_img): ## no use
    """https://towardsdatascience.com/computer-vision-for-beginners-part-4-64a8d9856208
    We can find the centroid of an image or calculate the area of a boundary field with the help of the notion called image moment. 
    What does a moment mean here? 
    The word ‘moment’ is a short period of time in common usage. 
    But in physics terminology, a moment is the product of the distance and another physical quantity meaning how a physical quantity is distributed or located. 
    So in computer vision, Image moment is how image pixel intensities are distributed according to their location. 
    It’s a weighted average of image pixel intensities and we can get the centroid or spatial information from the image moment.
    """

    ## convert the grayscale image to binary image
    ret,thresh = cv2.threshold(gray_img,127,255,0)
    ## calculate moments of binary image
    M = cv2.moments(thresh)

    ## calculate x,y coordinate of center
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])

    ## put text and highlight the center
    cv2.circle(subimg, (cX, cY), 5, (255, 255, 255), -1)
    cv2.putText(subimg, "centroid", (cX - 25, cY - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    ## display the image
    cv2.imshow("Image", subimg)
    cv2.waitKey()
    cv2.destroyAllWindows()


In [136]:
def circle_contour(subimg, gray_img): ## no use
    ## another way: build Gaussian threshold (refer 03_06.py)
    thresh = cv2.adaptiveThreshold(gray_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 115, 1)
    cv2.imshow("Binary", thresh)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    img2 = subimg.copy()
    index = -1
    thickness = 4
    color = (255, 0, 255)
    cv2.drawContours(img2, contours, index, color, thickness)
    cv2.imshow("Contours",img2)

    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [137]:
def circle_match(subimg): ## no use
    ## another way: mathTemplate (refer 04_03.py), collapse to work
    template = cv2.imread(path+'ball_template.jpg',0)

    cv2.imshow("Frame",subimg)
    cv2.imshow("Template",template)

    result = cv2.matchTemplate(subimg, template, cv2.TM_CCOEFF_NORMED)

    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
    print(max_val,max_loc)
    cv2.circle(result,max_loc, 15,255,2)
    cv2.imshow("Matching",result)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [138]:
def circle_Hough(img, gray_img):
    """ in a computing way
    another way: Circle detection (HoughCircles, refer to Learning OpenCV book)
    """
    bimg = cv2.medianBlur(gray_img, 5)
    cimg = cv2.cvtColor(bimg, cv2.COLOR_GRAY2BGR)

    circles = cv2.HoughCircles(bimg,cv2.HOUGH_GRADIENT,1,120, param1=100,param2=30,minRadius=0, maxRadius=0)
    circles = np.uint16(np.around(circles))

    def circle_detection(circles):
        "detect all the circles, plot circles + centers"
        print(circles, circles[0], circles[0,0])
        for i in circles[0,:]:
            # draw the outer circle
            print(i, i[0], i[1], i[2])
            cv2.circle(subimg,(i[0],i[1]),i[2],(0,255,0),2)
            # draw the center of the circle
            cv2.circle(subimg,(i[0],i[1]),2,(0,0,255),3)

    centroid = np.array([int(subimg.shape[0]/2), int(subimg.shape[1]/2)])
    dist = max(subimg.shape[0], subimg.shape[1])
    index = 0
    for i, c in enumerate(circles[0,:]):
        d = np.linalg.norm(np.array([c[0], c[1]])-centroid)
        # print(i, d)
        if d < dist:
            dist = d
            index = i

    circle = circles[0][index]
    print(index, circle)
    center, radius = circle[:2], circle[2]

    # draw the outer circle
    cv2.circle(subimg,center,radius,(0,0,255),2)
    # draw the center of the circle
    cv2.circle(subimg,center,2,(0,0,255),10)
    # print(circle[0],circle[1])

    cv2.imshow("Contours",subimg)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [139]:
if False:
    circle_Hough(img, gray_img)
else:
    "use above function to get a constant circle of the first photo"
    center, radius = np.array([244, 294]), 212
    
    # draw the outer circle
    cv2.circle(subimg,center,radius,(0,0,255),2)
    # draw the center of the circle
    cv2.circle(subimg,center,2,(0,0,255),10)
    # print(circle[0],circle[1])

    cv2.imshow("Contours",subimg)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## Compute sphere image information
Suppose it's a unit sphere. 
1. Given tilt-angle $\theta$ of the carema,  photos numbers $n$ 
2. Then computing big-circular-disk $D$ from the carema view, top-cusp-angle $\alpha=2\pi/n$ of the spherical patch $S$. Except the cusp point of $S$, the left can be parametrized by a regular mesh $M$. 
3. Project $M$ into $D$, corresponding planar patch $M_{2D}$ is the one that need to be chosen on every photo.
4. Smoothly transform the planar patch $M_{2D}$ into a rectangular patch $M_{rec}$, then neighbouring patches should smoothly transformed along the two rectangular straight lines.
5. Stitch the rectangular pathes together into 1 image.

In [140]:
### Set given angle and photo numbers
theta, n = (180-150.4)/180*np.pi, 8
alpha = 2*np.pi / n

In [141]:
### Read TOP, Bottom, Left-polyline-pts coodinate
import csv 

def read_csv(name):
    arrays = []
    with open(path0+name, 'r') as f:
        reader = csv.reader(f, delimiter=',') 
        for row in reader: 
            # print(row, len(row))
            row = str(row).replace(';', '')
            row = str(row).replace('[', '')
            row = str(row).replace(']', '')
            row = str(row).replace('\'', '')
            row = str(row).replace('\'', '')
            # print(row, type(row))
            if row =='Pi':
                row = str(np.pi)
            if row =='1.5 * Pi':
                row = str(1.5*np.pi)
            if row =='0.5 * Pi':
                row = str(0.5*np.pi)
            arrays.append(row)

    #print(arrays)
    arr = np.array(arrays[5:])
    phi = np.array([],dtype=float)
    for p in arr:
        phi = np.r_[phi, float(p)]
    print(name, phi.shape)
    return phi

phi_bottom = read_csv('\\phi_bottom.csv') + np.pi/2
phi_left = read_csv('\\phi_left.csv') + np.pi/2
rho_left = read_csv('\\rho_left.csv')


\phi_bottom.csv (21,)
\phi_left.csv (51,)
\rho_left.csv (51,)


In [142]:
### Plot the polyline
pts_bottom = np.c_[(radius * np.cos(phi_bottom)).astype(int), (radius * np.sin(phi_bottom)).astype(int)]

pts_left = np.c_[(radius * rho_left * np.cos(phi_left)).astype(int), (radius * rho_left * np.sin(phi_left)).astype(int)]

pts_right = np.c_[-pts_left[:,0], pts_left[:,1]]

# bdry_pts = center + np.vstack((pts_bottom[::-1], pts_left, pts_right))

In [143]:
### Extract the bounded patch
pts1 = (center + pts_left).reshape((-1, 1, 2))
pts2 = (center + pts_bottom).reshape((-1, 1, 2))
pts3 = (center + pts_right).reshape((-1, 1, 2))

isClosed = False
 
# Using cv2.polylines() method, Draw a Blue polygon with thickness of 1 px
ply1 = cv2.polylines(subimg, [pts1], isClosed, (255, 0, 0), thickness=2)
ply2 = cv2.polylines(subimg, [pts2], isClosed, (255, 0, 0), thickness=2)
ply3 = cv2.polylines(subimg, [pts3], isClosed, (255, 0, 0), thickness=2)
# Displaying the image
  
cv2.imshow(name, ply1)
cv2.imshow(name, ply2)
cv2.imshow(name, ply3)
cv2.waitKey()
cv2.destroyAllWindows()

In [144]:
### Extract cropped patch:

def crop_image(img, pts):
    # pts = np.array([[[10,150],[150,100],[300,150],[350,100],[310,20],[35,10]]])

    ## (1) Crop the bounding rectangular patch
    rect = cv2.boundingRect(pts)
    x,y,w,h = rect
    cropped = img[y:y+h, x:x+w].copy()

    ## (2) make mask
    pts = pts - pts.min(axis=0)

    mask = np.zeros(cropped.shape[:2], np.uint8)
    cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1, cv2.LINE_AA)

    ## (3) do bit-op
    dst = cv2.bitwise_and(cropped, cropped, mask=mask)

    ## (4) add the white background
    bg = np.ones_like(cropped, np.uint8)*255
    cv2.bitwise_not(bg,bg, mask=mask)
    dst2 = bg+ dst

    cv2.imshow("cropped" , cropped)
    cv2.imshow("white background" , dst2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


phi_bdry = read_csv('\\phi_bdry.csv') + np.pi/2
rho_bdry = read_csv('\\rho_bdry.csv')

pts_bdry = np.c_[(radius * rho_bdry * np.cos(phi_bdry)).astype(int), (radius * rho_bdry * np.sin(phi_bdry)).astype(int)]

pts_bdry = (center + pts_bdry).reshape((-1, 1, 2))

crop_image(subimg, pts_bdry)

\phi_bdry.csv (73,)
\rho_bdry.csv (73,)


In [145]:
## Transform the extracted patch into a regular rectangular patch
def warp_extracted_patch(subimg, pt_A, pt_B, pt_C, pt_D):
    # Create a copy of the image
    img_copy = np.copy(subimg)
    img_copy = cv2.cvtColor(img_copy,cv2.COLOR_BGR2RGB)

    # Here, I have used L2 norm. You can use L1 also.
    width_AD = np.sqrt(((pt_A[0] - pt_D[0]) ** 2) + ((pt_A[1] - pt_D[1]) ** 2))
    width_BC = np.sqrt(((pt_B[0] - pt_C[0]) ** 2) + ((pt_B[1] - pt_C[1]) ** 2))
    maxWidth = max(int(width_AD), int(width_BC))

    height_AB = np.sqrt(((pt_A[0] - pt_B[0]) ** 2) + ((pt_A[1] - pt_B[1]) ** 2))
    height_CD = np.sqrt(((pt_C[0] - pt_D[0]) ** 2) + ((pt_C[1] - pt_D[1]) ** 2))
    maxHeight = max(int(height_AB), int(height_CD))

    input_pts = np.float32([pt_A, pt_B, pt_C, pt_D])
    output_pts = np.float32([[0, 0],
                            [0, maxHeight - 1],
                            [maxWidth - 1, maxHeight - 1],
                            [maxWidth - 1, 0]])

    # Compute the perspective transform M
    M = cv2.getPerspectiveTransform(input_pts,output_pts)
    out = cv2.warpPerspective(subimg,M,(maxWidth, maxHeight),flags=cv2.INTER_LINEAR)

    cv2.imshow("Transformed patch" , out)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def get_pt(phi, rho, radius, center):
    phi += np.pi/2
    x = (radius * rho * np.cos(phi)).astype(int)
    y = (radius * rho * np.sin(phi)).astype(int)
    return np.array([x, y]) + center

pt_A = get_pt(5.320, 0.303, radius, center)
pt_B = get_pt(5.728,0.240, radius, center)
pt_C = get_pt(5.623,0.194, radius, center)
pt_D = get_pt(5.200, 0.265, radius, center)

warp_extracted_patch(subimg, pt_A, pt_B, pt_C, pt_D)

In [157]:
### Read the 5 lists of points of the extracted patch
def get_list_of_pts(phi, rho, radius, center):
    x = (radius * rho * np.cos(phi)).astype(int)
    y = (radius * rho * np.sin(phi)).astype(int)
    return np.c_[x, y] + center

phi = read_csv('\\patch_list1_phi.csv') + np.pi/2
rho = read_csv('\\patch_list1_r.csv')
pts_list1 = get_list_of_pts(phi, rho, radius, center)

phi = read_csv('\\patch_list2_phi.csv') + np.pi/2
rho = read_csv('\\patch_list2_r.csv')
pts_list2 = get_list_of_pts(phi, rho, radius, center)

phi = read_csv('\\patch_list3_phi.csv') + np.pi/2
rho = read_csv('\\patch_list3_r.csv')
pts_list3 = get_list_of_pts(phi, rho, radius, center)

phi = read_csv('\\patch_list4_phi.csv') + np.pi/2
rho = read_csv('\\patch_list4_r.csv')
pts_list4 = get_list_of_pts(phi, rho, radius, center)

phi = read_csv('\\patch_list5_phi.csv') + np.pi/2
rho = read_csv('\\patch_list5_r.csv')
pts_list5 = get_list_of_pts(phi, rho, radius, center)

num = len(pts_list1)
mid_ind = int(num/2)
width1 = np.linalg.norm(pts_list2[mid_ind]-pts_list1[mid_ind])
width2 = np.linalg.norm(pts_list3[mid_ind]-pts_list2[mid_ind])
width3 = np.linalg.norm(pts_list4[mid_ind]-pts_list3[mid_ind])
width4 = np.linalg.norm(pts_list5[mid_ind]-pts_list4[mid_ind])
width = int((width1+width2+width3+width4)/4)

hgt1 = np.linalg.norm(pts_list1[1:]- pts_list1[:-1], axis=1)
hgt2 = np.linalg.norm(pts_list2[1:]- pts_list2[:-1], axis=1)
hgt3 = np.linalg.norm(pts_list3[1:]- pts_list3[:-1], axis=1)
hgt4 = np.linalg.norm(pts_list4[1:]- pts_list4[:-1], axis=1)
hgt5 = np.linalg.norm(pts_list5[1:]- pts_list5[:-1], axis=1)
height = int(np.mean((hgt1+hgt2+hgt3+hgt4+hgt5)/5))
print(width, height)


\patch_list1_phi.csv (33,)
\patch_list1_r.csv (33,)
\patch_list2_phi.csv (33,)
\patch_list2_r.csv (33,)
\patch_list3_phi.csv (33,)
\patch_list3_r.csv (33,)
\patch_list4_phi.csv (33,)
\patch_list4_r.csv (33,)
\patch_list5_phi.csv (33,)
\patch_list5_r.csv (33,)
33 8


In [159]:
### Merge pieces of rectangular patches together to form 1 big rectangular patch
## Build the rectangular patch matrix

matrix = np.zeros((num* height, 5*width, 3), dtype=int)

output_pts = np.float32([[0, 0],
                        [0, height - 1],
                        [width - 1, height - 1],
                        [width - 1, 0]])

def strip(matrix, ptlist1, ptlist2, n_th):
    # Compute the perspective transform M
    Alist, Blist = ptlist1[1:][::-1], ptlist1[:-1][::-1] ## from the top, left-two
    Dlist, Clist = ptlist2[1:][::-1], ptlist2[:-1][::-1] ## from the top, right-two

    for i in range(num-1):
        pt_A, pt_B = Alist[i], Blist[i]
        pt_C, pt_D = Clist[i], Dlist[i]
        input_pts = np.float32([pt_A, pt_B, pt_C, pt_D])   
        M = cv2.getPerspectiveTransform(input_pts,output_pts)
        out = cv2.warpPerspective(subimg,M,(width, height),flags=cv2.INTER_LINEAR)
        matrix[i*height:(i+1)*height, n_th*width:(n_th+1)*width, :] = out
    return matrix
    
matrix = strip(matrix, pts_list1, pts_list2, 0)
matrix = strip(matrix, pts_list2, pts_list3, 1)
matrix = strip(matrix, pts_list3, pts_list4, 2)
matrix = strip(matrix, pts_list4, pts_list5, 3)

cv2.imshow("Rectangular Patch" , matrix)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0

error: OpenCV(4.8.0) D:/a/opencv-python/opencv-python/opencv/modules/highgui/src/precomp.hpp:155: error: (-215:Assertion failed) src_depth != CV_16F && src_depth != CV_32S in function 'convertToShow'


: 